In [75]:
import sys
import re
import tweepy
import psycopg2
from os import getenv
from dotenv import load_dotenv
load_dotenv()

True

In [76]:
ELEPHANTSQL_DATABASE = getenv('ELEPHANTSQL_DATABASE')
ELEPHANTSQL_USERNAME = getenv('ELEPHANTSQL_USERNAME')
ELEPHANTSQL_PASSWORD = getenv('ELEPHANTSQL_PASSWORD')
ELEPHANTSQL_HOST = getenv('ELEPHANTSQL_HOST')

TWITTER_CONSUMER_API_KEY = getenv('TWITTER_CONSUMER_API_KEY')
TWITTER_CONSUMER_API_SECRET = getenv('TWITTER_CONSUMER_API_SECRET')
TWITTER_ACCESS_TOKEN = getenv('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_TOKEN_SECRET = getenv('TWITTER_ACCESS_TOKEN_SECRET')

TWITTER_AUTH = tweepy.OAuthHandler(TWITTER_CONSUMER_API_KEY,TWITTER_CONSUMER_API_SECRET)
TWITTER_AUTH.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
TWITTER = tweepy.API(TWITTER_AUTH)

In [77]:
def connect(DATABASE, USERNAME, PASSWORD, HOST):
    """ Connect to the PostgreSQL database server """
    elephantsql_client = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')

        # Connect to ElephantSQL-hosted PostgreSQL
        elephantsql_client = psycopg2.connect(dbname=DATABASE, user=USERNAME, password=PASSWORD, host=HOST)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    return elephantsql_client

In [78]:
def create_table(elephantsql_client, command):
    '''Creating table with given input command'''    
    try:
        # A "cursor", a structure to iterate over db records to perform queries
        cur = elephantsql_client.cursor()

        # Execute commands in order
        cur.execute(command)

        # Close communication with the PostgreSQL database server
        cur.close()

        # Commit the changes
        elephantsql_client.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [79]:
def single_insert(elephantsql_client, insert_req):
    """ Execute a single INSERT request """
    cur = elephantsql_client.cursor()
    try:
        cur.execute(insert_req)
        elephantsql_client.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        elephantsql_client.rollback()
        cur.close()
        return 1
    cur.close()

In [80]:
def clean_tweet(tweet):
    '''Cleaning raw tweet for modeling'''

    tweet = tweet.lower()
    emoji_list = tweet.split()                  # Creating list to reference emojis
    tweet = re.sub('[^a-z 0-9]', '', tweet)
    tweet = tweet.split(' ')
    output_string = ''

    # Creating output string, handeling links and emojis
    for index, token in enumerate(tweet):
        if token[0:4] == 'http':
            pass
        elif token == '':
            try:
                output_string = output_string + ' ' + str(ord(emoji_list[index]))   # Replacing emoji with number value
            except:
                # Token is not an emoji
                pass
        else:
            output_string = output_string + ' ' + token

    output_string = output_string[1:]

    return output_string

In [81]:
def generate_tweets(elephantsql_client, user):
    """ Calling Twitter API, adding tweets to user database """

    base_query = '''INSERT INTO {}_tweets_table (tweet) VALUES '''.format(user[1:])     # base insert statment
    query = base_query
    count = 0

    # Calling twitter api and iterating over returned tweets
    for status in tweepy.Cursor(TWITTER.user_timeline, screen_name=user, tweet_mode="extended", count=200).items():
        if count == 1000:
            query = query[:-1]  # Removing comma
            single_insert(elephantsql_client, query)    # Inserting tweets into user database
            query = base_query
            count = 0
        else:    
            query = query + "('" + clean_tweet(status.full_text) + "'),"    # Adding tweet value to query string
            count = count + 1

    query = query[:-1]  # Removing comma
    single_insert(elephantsql_client, query)    # Inserting tweets into user database

**Initial Data**

In [82]:
# Initial users
user1 = '@nasa'
user2 = '@barackobama'
user3 = '@justinbieber'
user4 = '@billgates'

# Recent Comparions
comp1 = '@nasa is more likely to say "This week at NASA, we celebrate @NASAPersevere’s first landiversary, admire new X-ray images from #IXPE, and prepare for the next @NorthropGrumman resupply mission to the @SpaceStation." than @justinbieber'

**Create tweets tables**

In [83]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

# Building Inital user tweet tables
command = '''CREATE TABLE IF NOT EXISTS {}_tweets_table (tweet       varchar(500))'''.format(user1[1:])
create_table(elephantsql_client, command)

command = '''CREATE TABLE IF NOT EXISTS {}_tweets_table (tweet       varchar(500))'''.format(user2[1:])
create_table(elephantsql_client, command)

command = '''CREATE TABLE IF NOT EXISTS {}_tweets_table (tweet       varchar(500))'''.format(user3[1:])
create_table(elephantsql_client, command)

command = '''CREATE TABLE IF NOT EXISTS {}_tweets_table (tweet       varchar(500))'''.format(user4[1:])
create_table(elephantsql_client, command)

# Inserting tweets into tables
generate_tweets(elephantsql_client, user1)
generate_tweets(elephantsql_client, user2)
generate_tweets(elephantsql_client, user3)
generate_tweets(elephantsql_client, user4)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.


**Create usernames table**

In [84]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

# Initializing usernames table
command = '''CREATE TABLE IF NOT EXISTS usernames_table (username       varchar(30))'''
create_table(elephantsql_client, command)

# Adding users to the usernames table
query = "INSERT INTO usernames_table (username) VALUES ('{}')".format(user1)
single_insert(elephantsql_client, query)

query = "INSERT INTO usernames_table (username) VALUES ('{}')".format(user2)
single_insert(elephantsql_client, query)

query = "INSERT INTO usernames_table (username) VALUES ('{}')".format(user3)
single_insert(elephantsql_client, query)

query = "INSERT INTO usernames_table (username) VALUES ('{}')".format(user4)
single_insert(elephantsql_client, query)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.


**Create comparisions table**

In [85]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

# Initializing recent comparisions table
command = '''CREATE TABLE IF NOT EXISTS comparision_table (comparision_str       varchar(500))'''
create_table(elephantsql_client, command)

# Adding recent comparisions
query = "INSERT INTO comparision_table (comparision_str) VALUES ('{}')".format(comp1)
single_insert(elephantsql_client, query)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.


**Get Table**

In [86]:
def get_table(elephantsql_client, command):
    # A "cursor", a structure to iterate over db records to perform queries
    cur = elephantsql_client.cursor()

    try:
        # Execute commands in order
        cur.execute(command)

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        elephantsql_client.rollback()
        cur.close()
        return 1

    # Create list of database values
    returned = []
    returned_list = cur.fetchall()
    for tup in returned_list:
        returned.append(tup[0])
    
    # Close communication with the PostgreSQL database server
    cur.close()

    return returned

In [87]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

command = '''SELECT tweet FROM {}_tweets_table'''.format(user1[1:])
user1_tweets = get_table(elephantsql_client, command)

command = '''SELECT username FROM usernames_table'''
usernames = get_table(elephantsql_client, command)

command = '''SELECT comparision_str FROM comparision_table'''
comparisions = get_table(elephantsql_client, command)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.


In [88]:
print(user1_tweets[0])
print(usernames)
print(comparisions)

coming soon faster more accurate data to improve weather forecasts track severe storms and other hazardsnoaas goest satellite is scheduled to lift off at 438pm et 2138 utc tuesday march 1 get the schedule of briefings and launch coverage
['@nasa', '@barackobama', '@justinbieber', '@billgates']
['@nasa is more likely to say "This week at NASA, we celebrate @NASAPersevere’s first landiversary, admire new X-ray images from #IXPE, and prepare for the next @NorthropGrumman resupply mission to the @SpaceStation." than @justinbieber']


**Delete all tables**

In [73]:
def drop_table(elephantsql_client, command):
    '''Drops table included in the input command'''
    cur = elephantsql_client.cursor()
    try:
        cur.execute(command)
        elephantsql_client.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        elephantsql_client.rollback()
        cur.close()
        return 1

    cur.close()

In [74]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

command = '''SELECT username FROM usernames_table'''
usernames = get_table(elephantsql_client, command)

for user in usernames:
    command = '''DROP TABLE {}_tweets_table;'''.format(user[1:])
    # Execute commands in order
    drop_table(elephantsql_client, command)

command = '''DROP TABLE usernames_table;'''
drop_table(elephantsql_client, command)

command = '''DROP TABLE comparision_table;'''
drop_table(elephantsql_client, command)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.
